# Preload libraries and functions
First of all, let's import libraries that will be used in this example

In [12]:
# clear all variants
%reset -f   

# Autoreload the customize modules.
%reload_ext autoreload
# Or load_ext autoreload for the first run
%aimport HackModules.HackClasses, HackModules.HackFunctions 
%autoreload 1

import pandas as pd
import numpy as np
from sklearn import preprocessing as prep
import matplotlib.pyplot as plt

# import tensorflow as tf
# from tensorflow import keras

import HackModules.HackClasses as hc
import HackModules.HackFunctions as hf

#matplot inline
# np.set_printoptions(precision=8)

# help(hc) # To test the docstrings ((TBDBS To be deleted before submission))

# Alternative customize module import method
# import sys, os
# sys.path.append(os.getcwd()+'/HackModules/') # add /HackModules to the path
# import HackClasses as hc


Define some seldom used functions and class

In [2]:
%reset?

# Data Import and Initialization

In [ ]:
df = pd.read_excel('obs_data_w.xlsx', sheet_name=0) #sheet_name='Sheet1')
print(df)

## Train, cross validation and test data

### Random Method 1: Sample function of df

In [ ]:
temp=df.sample(frac=0.8) # randaom_state=200
test=df.drop(temp.index)
test=test.sample(frac=1) # shuffle the rows
train=temp.sample(frac=0.75)
cv=temp.drop(train.index)
print(train.shape)
print(cv.shape)
print(test.shape)
# train=data(train)
# cv=data(cv)
# test=data(test)
# print(train.array)

### Random Method 2: random array

In [ ]:
# idx=np.arange(0,len(df))
# np.random.shuffle(idx)
# temp_split=math.floor(0.8*len(df))
# #idx[0:20]
# train=df.iloc[idx[0:temp_split],:]
# test=df.iloc[idx[temp_split:],:] # Output all the elements after temp_split
# print(train.shape)
# print(test.shape)

## Split the data into data and results

In [ ]:
train_data=train.loc[:,['V','T']]
train_unc=train.uncertainty.values
train_J=train.J.values

cv_data=cv.loc[:,['V','T']]
cv_unc=cv.uncertainty.values
cv_J=cv.J.values

test_data=test.loc[:,['V','T']]
test_unc=test.uncertainty.values
test_J=test.J.values

train=data(train_data)
cv=data(cv_data)
test=data(test_data)

# print(test_data)
# J=J.tolist()

## Add the polynominal features

In [ ]:
# train data
train_data_poly = train.polyFeature(2)
print('train_data = \n', train_data_poly, '\n')  # print(f'train_data=\n{train_data_array}')

# cv data
cv_data_poly=cv.polyFeature(2)
print('cross validation = \n', cv_data_poly, '\n')

# test data
test_data_poly=test.polyFeature(2)
print('test data = \n', test_data_poly, '\n')


## Data Normalization (Two methods, range or stand deviation)  

(__We should try both__)

In [ ]:
# Normalization method
method='std'

# # train data
# train_data_norm=normalizeFeature(train_data_poly, method)
# print('normalized train_data = \n', train_data_norm, '\n')

# # cross valiation data
# cv_data_norm=normalizeFeature(cv_data_poly, method)
# print('normalized cv_data = \n', cv_data_norm, '\n')

# # test data
# test_data_norm=normalizeFeature(test_data_poly, method)
# print('normalized test_data = \n', test_data_norm, '\n')

#
# train_data_norm_class=train.polyFeature(3)
train_data_norm_class=train.normalizeFeature('std')
print(train_data_norm_class)

# Model Training

## Method 1: Normal Equation

### Training without regulation

In [ ]:
theta=normalEq(train_data_norm, train_J) 
print(f'theta = {theta}')
error_train=computeCost(train_data_norm, train_J, theta)
error_test=computeCost(test_data_norm, test_J, theta)

print('The training error is ', error_train)
print('The test error is ', error_test)

# OTHER PRINT EXPRESSION WITH FORMAT
# print('The training error is %.10f'%error)
# print('error = {:.10f}'.format(error))
# print(f'error = {error}')
# print('error= ', error)

### Training with regulation

In [ ]:
featureSize=train_data_norm.shape[1]
beta_range=np.geomspace(0.001,50, num = 200, endpoint = True, dtype=np.float64)
beta_range=np.insert(beta_range,0,0) # insert a zero element in the first index
theta_reg=np.zeros((beta_range.size, featureSize))
error_train_reg=np.zeros(beta_range.size)
error_cv_reg=np.zeros(beta_range.size)
for index, beta in enumerate(beta_range):
    theta_reg[index]=normalRegEq(train_data_norm, train_J, beta)
    error_train_reg[index]=computeCost(train_data_norm, train_J, theta_reg[index])
    error_cv_reg[index]=computeCost(cv_data_norm, cv_J, theta_reg[index])

# plot error vs. beta
plt.figure()
plt.plot(beta_range, error_cv_reg, label = 'Val error')
plt.plot(beta_range, error_train_reg, label = 'Train error')
plt.xlabel('regulization coefficient (beta)')
plt.ylabel('error')
plt.legend()

idx=error_cv_reg.argmin()
beta_best = beta_range[idx]
theta_best = theta_reg[idx]
error_test_reg=computeCost(test_data_norm, test_J, theta_best)

print('The best value of THETA is ', theta_best)
print('The best value of BETA is ', beta_best)
print('The test error is ', error_test_reg)

# Prediction

In [ ]:
train_data.shape[1]

In [ ]:
t=data(test_data)

In [ ]:
print(t.array)

In [ ]:
t.polyFeature(2)
t.array.shape

In [ ]:
type(theta_reg)